### The logic of the notebook

In order to be able to perform multiple operations I have designed the system as follows:

- first, I create the most basic function which just updates the owner of a player
- second, the function to update the transfer database is created
- then I defined the class transfer, with argument name of the player
    - the class transfer can perform all market operations and then save database entries
    - however, it is not convenient to save directly to the database, in case there are accidental mistakes
- so I created another class called market session:
    - first, you start a market session and create each transfer within the session using the inner class transfer
    - when you are done with all the transfers, and are sure there are no mistakes you confirm the session
    - if you notice a mistake you can cancel the session and the databases rest unchanged

- finally, everything is managed using widgets, so there is no need to write lines of code to perform operations
    
## OPEN QUESTIONS:

- what is the cost for draft? I set it to 0 for now
- is this the best way to define exchanges?
- we could in principle include in the player's info last transfer operation's id, but it is enough to look at the most recent in the transfer db

# Procedure:
- 1) run all the support code, it defines all the functions and needed classes
- 2) run the start session cell
- 3) click start session (if a session is in progress this will be continued)
- 4) run all the sessions below that
- 5) start to perform all the needed operations
- 6) at the end of the market operations confirm session by clicking on the button 

In [1]:
from IPython.display import HTML, clear_output

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### SUPPORT CELLS: run this first
Click Delete button only if you want to discard all the info of the ongoing market session

In [3]:
from __future__ import print_function

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import NoSuchElementException   


import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(ChromeDriverManager().install())


options = Options()
options.headless = True
options.add_argument("--window-size=1920,10200") #this is important, to tell it how much of the webpage to import
driver = webdriver.Chrome(options=options, executable_path=r'/usr/local/bin/chromedriver')

import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import ipywidgets as widgets
import time
import os

with open('credential.json','r') as f:
    cred = json.load(f)

cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]


def scarica_quot(stagione ='2020-21'):
    '''uses selenium to download the updated stats for the players'''
    
    
    driver.get('https://www.fantacalcio.it/quotazioni-fantacalcio')
    button = driver.find_element_by_id("toexcel")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(5)
    for file in os.listdir('./'):
        if 'Quotazioni_Fantacalcio' in file:
            data = pd.read_excel(file,skiprows=[0])
            data.index = list(range(len(data)))
            os.remove(file)
    return data

def quotation_dict(stagione ='2020-21'):
    '''uses selenium to download the updated quotations for the players'''
    
    
    quot = scarica_quot(stagione)
    Names = quot['Nome']
    quot_a = quot['Qt. A']
    dict_quot = {}
    for idx in range(len(Names)):
        dict_quot[Names[idx]]=quot_a[idx]
    return dict_quot



def update_owner(new_owner, mode, squad, dict_quot, cost = 1, loan = False, move_up = False, name = False, Id = False, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    """
Function documentation:
This is the general function to update the owner of a player.
Arguments:

- new_owner: string, the new owner of the team (pietro, luca,...)
- mode: string, the acquisition mode (Scambio, draft,...)
- squad: assignment in the new team (main or primavera)
- name/id: string, use one OR the other as a filter to query the database
- rest: details to access database, set to default values

Note: I'm planning to use this function inside higher level functions, which is why I have the timestamp option, 
if you are doing tests there is no reason to update the date, but if you include this in a market operation function
(for instance exchange players) you want to register this and the higher level function will also create the database entry

"""
    
    
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    today = date.today()
    d1 = today.strftime("%Y/%m/%d")
    
    if name in dict_quot.keys():
        quot_a = dict_quot[name]
    else:
        quot_a = abs(cost)
    
    if name:
        filt = {'name': name}
    elif Id:
        filt = {'_id': str(Id)}
    else:
        print('Please enter name or Id argument')
        
        return None
        
    dic = collection.find_one(filt)
    old_owner = dic['info']['contract']['owner'] #when database structure changes, Modify this and following accordingly
    
    if dic['info']['current_team']['owner'] is None:
        old_team = None
    else:
        old_team = dic['info']['current_team']['owner'] +', ' + dic['info']['current_team']['squad']
    
    if loan:

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        collection.update_one(filt, {'$set':{'info.current_team.quotation_initial': int(quot_a)}}) # dict_quot[name]}})
        collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            
    
    elif move_up:
        
        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        collection.update_one(filt, {'$set':{'info.current_team.quotation_initial': int(quot_a)}})
        
        
        collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
        
        
        
    else: #draft, cash, auction, exchange
    
        collection.update_one(filt, {'$set':{'info.contract.acquisition_mode': mode}})
        collection.update_one(filt, {'$set':{'info.contract.previous_owner': old_owner}})
        collection.update_one(filt, {'$set':{'info.contract.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.contract.quotation_initial': int(quot_a)}})

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        collection.update_one(filt, {'$set':{'info.current_team.quotation_initial': int(quot_a)}})
        
        collection.update_one(filt, {'$set':{'info.contract.start_date': d1}})
        collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) 
        

def update_Tr_DB(transfer_data, CR = cred['cred'], DB = 'Game', COT = 'Transfers'):
    '''this function is similar to the update_owner one, it is used to update the transfers database. The difference is that
    all the data to be stored is passed directly to the function as a dictionary and is stored as is on mongodb, only added
    thing is the unique id, which is a timestamp
    danger!!! Timestamp is local (US for the first operations) so it might cause error if operations are performed in multiple 
    timezones!!
    '''
    
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[COT]
    
    now = datetime.datetime.today()
    Id = now.strftime("%Y%m%d%H%M%S")
    #d1 = now.strftime("%Y/%m/%d")
    
    dic0 = transfer_data[0]
    
    if dic0['operation'] in ['Asta', 'Draft', 'Acquisto', 'Prestito', 'Fine Prestito', 'Promozione', 'Svincolo']:
        dic0['_id'] = Id
        #dic0['date'] = d1
        
        collection.insert_one(dic0)
    
    elif dic0['operation'] == 'Scambio':
        
        dic1 = transfer_data[1]
        
        
        dic0['_id'] = Id
        #dic0['date'] = d1
        #dic1['date'] = d1
        dic1['_id'] = str(int(Id) + 1)
        
        collection.insert_one(dic0)
        collection.insert_one(dic1)
    




class transfer:
    
    """
This is an inner level class, it performs all the market operations and creates database entries
But we'll use it inside another class, so all operations need to be confirmed before being saved

"""
    
    #need to add quotation info everywhere, maybe as return of transf_db

    def __init__(self, name, dict_quot, CR, DB, CO, COT, CO_Man):
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        
        self.collection = db[CO]
        self.collection_man = db[CO_Man]
        self.name = name
        self.info = collection.find_one({'name': name})
        self.CC = (CR, DB, CO)
        self.COT = COT
        self.owner = self.info['info']['contract']['owner']
        self.previous_squad = self.info['info']['current_team']['squad']
        
        self.dict_quot = dict_quot
        
        today = date.today()
        self.d1 = today.strftime("%Y/%m/%d")
         
        
    def draft_in(self, new_owner, squad, cost = 1):
        '''used to draft a player during the summer session'''
        (CR, DB, CO) = self.CC
        
        collection_man = self.collection_man
        man_data = collection_man.find_one({'owner': new_owner})
        
        budget = man_data['budget']
        
        if cost > budget:
            return False
        else:
            
            #print(new_owner+' has acquired '+self.name+' through draft.\nAssigned to '+squad+' squad\n')
            
            
            
            update_owner(new_owner = new_owner, mode = 'Draft', squad = squad, dict_quot = self.dict_quot, name = self.name, CR = CR, DB = DB, CO = CO)

            self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
    
            quot_a = self.dict_quot[self.name]
            
            date_num = self.d1.split('/')
            date_num = int(date_num[0]+date_num[1]+date_num[2])
            transfer_data = [{
                'name': self.name,
                'operation': 'Draft',
                'previous_owner': None,
                'new_owner': new_owner,
                'squad': squad,
                'previous_squad': None,
                'cost': cost,
                'date': self.d1,
                'date_num':date_num,
                'quotation_to_date': int(quot_a)
                          }]


            update_Tr_DB(transfer_data, CR, DB, self.COT)
            self.collection_man.update_one({'owner': new_owner}, {'$inc': {'budget': - cost}})
            #transf_db_update() #function to be defined
        
            return True
        
        
    
    def auction_in(self, new_owner, squad, cost):
        '''used to assign player with auctions'''
        (CR, DB, CO) = self.CC
        
        collection_man = self.collection_man
        man_data = collection_man.find_one({'owner': new_owner})
        
        budget = man_data['budget']
        
        if cost > budget:
            return False
        else:
            
            
            update_owner(new_owner = new_owner, mode = 'Asta', squad = squad, dict_quot = self.dict_quot, name = self.name, CR = CR, DB = DB, CO = CO)

            self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
            #transf_db_update() #function to be defined, includes budgets adjustments
            
            quot_a = self.dict_quot[self.name]
            
            date_num = self.d1.split('/')
            date_num = int(date_num[0]+date_num[1]+date_num[2])
            transfer_data = [{
                'name': self.name,
                'operation': 'Asta',
                'previous_owner': None,
                'new_owner': new_owner,
                'squad': squad,
                'previous_squad': None,
                'cost': cost,
                'date': self.d1,
                'date_num':date_num,
                'quotation_to_date': int(quot_a)
                          }]


            update_Tr_DB(transfer_data, CR, DB, self.COT)
            self.collection_man.update_one({'owner': new_owner}, {'$inc': {'budget': - cost}})
            return True
        
        
    
    def cash_in(self, new_owner, squad, cost):
        '''used to purchase a player from another manager'''
        old_owner = self.info['info']['contract']['owner']
        (CR, DB, CO) = self.CC
        
        collection_man = self.collection_man
        
        man_data = collection_man.find_one({'owner': new_owner})
        
        budget = man_data['budget']
        
        if cost > budget:
            return False
        else:
            update_owner(new_owner = new_owner, mode = 'Acquisto', squad = squad, dict_quot = self.dict_quot, name = self.name, CR = CR, DB = DB, CO = CO)

            self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
            #transf_db_update() #function to be defined, includes budgets adjustments

            quot_a = self.dict_quot[self.name]
            
            date_num = self.d1.split('/')
            date_num = int(date_num[0]+date_num[1]+date_num[2])
            transfer_data = [{
                'name': self.name,
                'operation': 'Acquisto',
                'new_owner': new_owner,
                'previous_owner': self.owner,
                'old_owner': old_owner,
                'squad': squad,
                'previous_squad': self.previous_squad,
                'cost': cost,
                'date': self.d1,
                'date_num':date_num,
                'quotation_to_date': int(quot_a)
                          }]


            update_Tr_DB(transfer_data, CR, DB, self.COT)
            self.collection_man.update_one({'owner': new_owner}, {'$inc': {'budget': - cost}})
            
            return True
        
        
    
    def exchange_with(self, exch_player, cost, squad, squad_exch_pl):
        '''used to perfor  exchanges'''
        
        #NEED TO ADD THE BUDGET CONTROL OPERATION, BUT NEEDS TO BE DONE CAREFULLY IN CASE OF NEGATIVE BUDGET
        
        (CR, DB, CO) = self.CC
        collection = self.collection
        self.exch_player = exch_player
        
        #self.name is the name of the player which was chosen to be transfered 
        #
        
        exch_info = collection.find_one({'name': exch_player})
        ex_cost = exch_info['info']['contract']['cost']
        self_cost = self.info['info']['contract']['cost']
        
        
        exch_owner = exch_info['info']['contract']['owner']
        owner = self.info['info']['contract']['owner']
        
        collection_man = self.collection_man
        
        man_data = collection_man.find_one({'owner': owner})
        man_data_ex = collection_man.find_one({'owner': exch_owner})
        
        budget_own = man_data['budget']
        budget_ex = man_data_ex['budget']
        
        #I think this is correct but double check after trial
        if - cost > budget_own or  cost > budget_ex:
            return False
        else:
        
            
            
            update_owner(new_owner = exch_owner, mode = 'Scambio', squad = squad, dict_quot = self.dict_quot, name = self.name, CR = CR, DB = DB, CO = CO)
            update_owner(new_owner = owner, mode = 'Scambio', squad = squad_exch_pl, dict_quot = self.dict_quot, name = exch_player, CR = CR, DB = DB, CO = CO)
            
            
            #here the cost that is recorded includes the cost paid by the previous team for the player.
            
            self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': int(cost)+int(ex_cost)}})
            self.collection.update_one({'name': exch_player},{'$set':{'info.contract.cost': -int(cost)+int(self_cost)}})

            quot_a = self.dict_quot[self.name]
            quot_a_x = self.dict_quot[exch_player]
            
            date_num = self.d1.split('/')
            date_num = int(date_num[0]+date_num[1]+date_num[2])
            transfer_data_0 = {
                'name': self.name,
                'operation': 'Scambio',
                'previous_owner': owner,
                'new_owner': exch_owner,
                'exchange_player': exch_player,
                'squad': squad,
                'previous_squad': squad_exch_pl,
                'cost': cost, #ADD ORIGINAL COST OF exch_player (not here, )
                'date': self.d1,
                'date_num':date_num,
                'quotation_to_date': int(quot_a)
                          }
            transfer_data_1 = {
                'name': exch_player,
                'operation': 'Scambio',
                'previous_owner': exch_owner,
                'new_owner': owner,
                'exchange_player': self.name,
                'squad': squad_exch_pl,
                'previous_squad': squad,
                'cost': -cost, #ADD ORIGINAL COST OF self.name (not here, )
                'date': self.d1,
                'date_num':date_num,
                'quotation_to_date': int(quot_a_x)
                          }
            
            
            
            transfer_data = [transfer_data_0, transfer_data_1]

            update_Tr_DB(transfer_data, CR, DB, self.COT)
            self.collection_man.update_one({'owner': exch_owner}, {'$inc': {'budget': - cost}})
            self.collection_man.update_one({'owner': owner}, {'$inc': {'budget': cost}})
            
            return True
        
    
    def loan_to(self, owner, cost, squad, end_season = None, final_fee = None):
        '''used to perform loans'''
        (CR, DB, CO) = self.CC

        collection = self.collection
        collection_man = self.collection_man
        man_data = collection_man.find_one({'owner': new_owner})
        
        budget = man_data['budget']
        
        if cost > budget:
            return False
        else:
            update_owner(new_owner = owner, mode = None, squad = squad, dict_quot = self.dict_quot, loan = True, name = self.name, CR = CR, DB = DB, CO = CO)


            date = datetime.date.today()

            m, y = date.month, date.year
            if m > 7:
                y = y + 1

            loan_info = {
            'start_date': self.d1,
            'expire_date': str(y)+'/07/31', #all loans expire the next July 31st
            'cost': cost,
            'redemption_right': end_season, #None, Option or Must
            'redemption_cost': final_fee, # None or >0
            }

            collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': True}}) 
            collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': loan_info}}) 
            
            quot_a = self.dict_quot[self.name]
            
            date_num = self.d1.split('/')
            date_num = int(date_num[0]+date_num[1]+date_num[2])
            transfer_data = [{
                'name': self.name,
                'operation': 'Prestito',
                'previous_owner': self.owner,
                'new_owner': new_owner,
                #'new_team': owner +', '+ squad,
                'squad': squad,
                'previous_squad': self.previous_squad,
                'loan_info': loan_info,
                'date': self.d1,
                'date_num':date_num,
                'cost': cost,
                'quotation_to_date': int(quot_a)
                          }]
            


            update_Tr_DB(transfer_data, CR, DB, self.COT)
            self.collection_man.update_one({'owner': new_owner}, {'$inc': {'budget': - cost}})
            self.collection_man.update_one({'owner': self.owner}, {'$inc': {'budget': cost}})
            
            return True
        
        
    
    def end_loan(self, squad, redeemed = False):
        '''used to end loans'''
        (CR, DB, CO) = self.CC
        collection = self.collection
        
        dic = self.info
        owner = dic['info']['contract']['owner']
        owner_loan = self.info['info']['current_team']['owner']
        
        update_owner(new_owner = owner, mode = None, squad = squad, dict_quot = self.dict_quot, loan = True, name = self.name, CR = CR, DB = DB, CO = CO)
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': False}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': None}}) 
        
        quot_a = self.dict_quot[self.name]
        
        date_num = self.d1.split('/')
        date_num = int(date_num[0]+date_num[1]+date_num[2])
            
        transfer_data = [{
            'name': self.name,
            'operation': 'Fine Prestito',
            #'new_team': owner +', '+ squad,
            'previous_owner': owner_loan,
            'new_owner': owner,
            'squad': squad,
            'previous_squad': self.previous_squad,
            'cost': None,
            'redeemed': redeemed,
            'date': self.d1,
            'date_num':date_num,
            'quotation_to_date': int(quot_a)
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        if redeemed:
            
            print('Registrare operazione di trasferimento!!!')
        
        return True
        
        
    def move_up(self):
        '''used to move players up to main squad from primavera'''
        (CR, DB, CO) = self.CC
        update_owner(new_owner = None, mode = None, squad = 'main', dict_quot = self.dict_quot, move_up = True, name = self.name,  CR = CR, DB = DB, CO = CO)
        
        quot_a = self.dict_quot[self.name]
        
        
        date_num = self.d1.split('/')
        date_num = int(date_num[0]+date_num[1]+date_num[2])
            
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Promozione',
            'new_owner':self.owner,
            'previous_owner':self.owner,
            'squad': 'main',
            'previous_squad': 'primavera',
            'date': self.d1,
            'date_num':date_num,
            'cost': None,
            'quotation_to_date': int(quot_a) 
                      }]
        
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        return True
        
    def rescind(self, cost):
        '''used to rescind a player contract'''
        (CR, DB, CO) = self.CC
        update_owner(new_owner = None, mode = 'Svincolo', squad = None, cost = cost, dict_quot = self.dict_quot, name = self.name, CR = CR, DB = DB, CO = CO)
        
        if self.name in self.dict_quot.keys():
            quot_a = self.dict_quot[self.name]
        else:
            quot_a = cost #self.dict_quot[self.name]
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': abs(cost)}})
       
        
        date_num = self.d1.split('/')
        date_num = int(date_num[0]+date_num[1]+date_num[2])
            
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Svincolo',
            'new_owner': None,
            'previous_owner':self.owner,
            'squad': None,
            'previous_squad': self.previous_squad,
            'date': self.d1,
            'date_num':date_num,
            'quotation_to_date': int(quot_a),
            'cost': abs(cost)
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        self.collection_man.update_one({'owner': self.info['info']['contract']['owner']}, {'$inc': {'budget': abs(cost)}})
        
        return True



class market_session:
    
    """
This second layer is designed to prevent accidental changes in the database:
First all the players collection is duplicated to a temp one
then all the changes are made in the temp

"""
    
    def __init__(self, dict_quot, CR = cred['cred'], DB = 'Game', CO = 'Players', COT = 'Transfers', CO_MAN = 'Managers'):
        #copy entire database into tempPlayers
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        collection_man = db[CO_MAN]
        COt = 'tempPlayers'
        CO_MANt = 'tempManagers'
        
        collectionT = db[COt]
        collection_manT = db[CO_MANt]
        
        if len(list(collectionT.find({}))) > 0 or len(list(collection_manT.find({}))) > 0:
            print('Session Continued...', end="\n")
        
        else:
            posts = list(collection.find({}))
            collectionT.insert_many(posts)

            posts_man = list(collection_man.find({}))
            collection_manT.insert_many(posts_man)
        
        self.CC = (CR, DB, COt)
        self.CO = CO
        self.COTt = 'tempTransfers'
        self.CO_MANt = 'tempManagers'
        self.COT = COT
        self.CO_MAN = CO_MAN
        
        self.dict_quot = dict_quot
        
    def select_pl(self, name):
        (CR, DB, COt) = self.CC
        dict_quot = self.dict_quot
        self.player = transfer(name, dict_quot, CR, DB, COt, self.COTt, self.CO_MANt)
    
    def confirm_session(self):
        #here we span through both temp collections (players and transfers) and replace/add entries to the official database
        #then we erase the temp databases
        (CR, DB, COt) = self.CC
        CO = self.CO
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        collectionT = db[COt]
        posts = list(collectionT.find({}))
        
        if len(posts)>0:
            collection.delete_many({})
            collection.insert_many(posts)
            collectionT.delete_many({})
        
        coll_tr = db[self.COT]
        coll_trT = db[self.COTt]
        posts = list(coll_trT.find({}))
        
        
        if len(posts)>0:
            coll_tr.insert_many(posts)
            coll_trT.delete_many({})
        
        coll_man = db[self.CO_MAN]
        coll_manT = db[self.CO_MANt]
        posts = list(coll_manT.find({}))
        if len(posts)>0:
            coll_man.delete_many({})
            coll_man.insert_many(posts)
            coll_manT.delete_many({})
        
        
    def cancel_session(self):
        #erase temporary database: need to redefine market session
        (CR, DB, COt) = self.CC
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collectionT = db[COt]
        coll_trT = db[self.COTt]
        coll_manT = db[self.CO_MANt]
        
        collectionT.delete_many({})
        coll_trT.delete_many({})
        coll_manT.delete_many({})

def player(name, session):
    #select one player and assign it to a variable 
    session.select_pl(name)
    return session.player #returns a class transfer object

btn_delete_db = widgets.Button(description='Delete all temporary DBs',layout=widgets.Layout(width='200px', height='40px'))

def delete_db_event(obj):
    db['tempTransfers'].delete_many({})
    db['tempPlayers'].delete_many({})
    db['tempManagers'].delete_many({})

btn_delete_db.on_click(delete_db_event)
btn_delete_db.style.button_color = 'red'
display(btn_delete_db)

Button(description='Delete all temporary DBs', layout=Layout(height='40px', width='200px'), style=ButtonStyle(…

# MARKET OPERATIONS DASHBOARD

### Session start/confirm/backups
Run Start/Continue at the beginning of the given session (e.g. summer session). If a session is ongoing, click to be able to continue with operations

- quotation dictionary is imported for 2020/21 season for now, change input in following seasons

NOTE: YOU CAN CLICK THE RED BUTTON ABOVE TO OVERRIDE ALL THE ACTIONS AND DELETE THE tempDB (NOT THE BACKUP)

The two backup buttons create entries with timestamps for
- entire temp DB (orange)
- entire current DB (without the changes of the ongoing session, to backup those confirm first)

In [4]:
btn_start = widgets.Button(description='Start/Continue Session',layout=widgets.Layout(width='200px', height='40px'))

btn_cancel = widgets.Button(description='Cancel Session',layout=btn_start.layout)

btn_confirm = widgets.Button(description='Confirm Session',layout=btn_start.layout)

btn_temp_backup = widgets.Button(description='BackUp temporary DBs',layout=btn_start.layout)

btn_backup = widgets.Button(description='BackUp full DBs',layout=btn_start.layout)


def sess_start_event(obj):
    print('WAIT!!!                     ', end="\r", flush=True)
    global sess
    dict_quot = quotation_dict(stagione ='2020-21')
    sess = market_session(dict_quot)
    print('Session Started!             ', end="\r", flush=True)#.format(obj.description))
def sess_cancel_event(obj):
    
    sess.cancel_session()
    print('Session Canceled!              ', end="\r", flush=True)#.format(obj.description))
    
def sess_confirm_event(obj):
    sess.confirm_session()
    print('Session Confirmed!           ', end="\r", flush=True)#.format(obj.description))

def create_temp_backup_event(obj):
    now = datetime.datetime.today()
    Id = now.strftime("%Y%m%d%H%M%S")
    
    transfers = db['tempTransfers'].find({})
    players = db['tempPlayers'].find({})
    managers = db['tempManagers'].find({})
    
    dic = {
        '_id': Id,
        'transfers': list(transfers),
        'players': list(players),
        'managers': list(managers),
        'date': now.strftime('%Y/%m/%d'),
        'time': now.strftime('%H-%M-%S')
    }
    
    back_db = cluster['Game_Backup']
    back_db['tempDB_Backups'].insert_one(dic)
    print('Temporary DBs Backed up!          ', end="\r", flush=True)#.format(obj.description))
    
def create_backup_event(obj):
    now = datetime.datetime.today()
    Id = now.strftime("%Y%m%d%H%M%S")
    
    transfers = db['Transfers'].find({})
    players = db['Players'].find({})
    managers = db['Managers'].find({})
    
    dic = {
        '_id': Id,
        'transfers': list(transfers),
        'players': list(players),
        'managers': list(managers),
        'date': now.strftime('%Y/%m/%d'),
        'time': now.strftime('%H-%M-%S')
    }
    
    back_db = cluster['Game_Backup']
    back_db['DB_Backups'].insert_one(dic)
    print('Full DBs Backed up!          ', end="\r", flush=True)#.format(obj.description))
    
btn_start.on_click(sess_start_event)
btn_start.style.button_color = 'skyblue'
btn_cancel.on_click(sess_cancel_event)
btn_cancel.style.button_color = 'lightcoral'  
btn_confirm.on_click(sess_confirm_event)
btn_confirm.style.button_color = 'lightgreen'  

btn_temp_backup.on_click(create_temp_backup_event)
btn_temp_backup.style.button_color = 'orange'

btn_backup.on_click(create_backup_event)
btn_backup.style.button_color = 'yellow'

#I removed the cancel button because now we want to be able to continue performing market operation across several days

input_start = widgets.HBox([btn_start, btn_confirm, btn_temp_backup, btn_backup])
display(input_start)


### Selection of the  player to be transfered
RUN THE CELL ONLY AFTER SESSION START
Note: Players are listed by OWNER, so those on loan are not under the team they play with

In [5]:
#owners = ['Choose one','Svincolati','luca','pietro','enzo','nanni','mario','emiliano','franky','musci8']
owners = ['Svincolati','luca','pietro','enzo','nanni','mario','emiliano','franky','musci8']

def owner_players(owner):
    if owner == 'Svincolati':
        results = db['tempPlayers'].find({'info.contract.owner': None})
        names = []
        for result in results:
            names.append(result['name'])
    elif owner == 'Choose one':
        names = ['Choose owner first']
    else:
        results = db['tempPlayers'].find({'info.contract.owner': owner})
        names = []
        for result in results:
            names.append(result['name'])
    names.sort()
    return names

def confirm_player(Name):
    global pl
    pl = player(Name, sess)
    if Name == 'Choose owner first':
        print('')
    else:
        print('The player selected to be transferred is ' + Name+'                     ')
    

def selection(pl_owner):
    global G_owner
    G_owner = pl_owner
    lista.options = owner_players(pl_owner)



scW = widgets.Dropdown(options=owners, description = 'Owner')

init = scW.value

lista = widgets.Dropdown(options=owner_players(init), description = 'Player')

j = widgets.interactive(confirm_player, Name=lista)

i = widgets.interactive(selection, pl_owner=scW)





# new_owner, squad, cost, exch_player, squad_exch_pl, end_season, final_fee


#display(i)
#display(j)

btn_head_pl = widgets.Button(description='Select Player to Transfer',layout=widgets.Layout(width='350px', height='40px'))

btn_head_pl.style.button_color = 'skyblue'


input_pl = widgets.VBox([btn_head_pl,i,j])
display(input_pl)


## Selection of parameters


Use only the ones that are relevant to the market operation

In [6]:
cost = 0
New_owner = widgets.Dropdown(options =    owners, description = 'New Owner')

def new_owner_eventhandler(change):
    global new_owner
    new_owner = change.new
    print('After operation '+pl.name+' will be owned by ' + new_owner+'                               ', end="\r", flush=True)

New_owner.observe(new_owner_eventhandler, names = 'value')

#display(New_owner)

squads = widgets.Dropdown(options = ['Choose', 'main', 'primavera'], description = 'New Squad')

def squad_eventhandler(change):
    global squad
    squad = change.new
    print('After operation '+pl.name+' will be in ' + new_owner+'\'s '+squad+' squad                       ', end="\r", flush=True)

squads.observe(squad_eventhandler, names = 'value')

#display(squads)

cost_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Cost')

def cost_num_eventhandler(change):
    global cost
    cost = change.new
    if cost >= 0:
        print(new_owner+' will pay ' + str(change.new)+' FM for the operation involving '+pl.name+'                              ', end="\r", flush=True)
    else:
        print(new_owner+' will receive ' + str(- change.new)+' FM for the operation involving '+pl.name+'                          ', end="\r", flush=True)
        
cost_num.observe(cost_num_eventhandler, names='value')


End_season = widgets.Dropdown(options = ['Choose', None, 'Option', 'Must'], description = 'End Choice')

def End_season_eventhandler(change):
    global end_season
    end_season = change.new
    if end_season is None:
        print('If loan is chosen, no option is selected for end of the season                                         ', end="\r", flush=True)
    elif end_season is 'Option':
        print('If loan is chosen, '+pl.name+' CAN be redeemed at final fee value                                     ', end="\r", flush=True)
    else:
        print('If loan is chosen, '+pl.name+' MUST be redeemed at final fee value                                    ', end="\r", flush=True)

End_season.observe(End_season_eventhandler, names = 'value')



fee_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Final fee')

def fee_num_eventhandler(change):
    global final_fee
    final_fee = change.new
    if cost >= 0:
        print(new_owner+' can/will pay ' + str(final_fee)+' to redeem '+pl.name+' at the end of the season              ', end="\r", flush=True)
        
fee_num.observe(fee_num_eventhandler, names='value')

Redeemed = widgets.Dropdown(options = ['Choose', True,False], description = 'Redeemed')

def End_loan_eventhandler(change):
    global redeemed
    redeemed = change.new
    if redeemed == True:
        print('If loan is chosen, '+pl.name+' will be redeemed.                 ', end="\r", flush=True)
    else:
        print('If loan is chosen, '+pl.name+' will not be redeemed.                  ', end="\r", flush=True)

Redeemed.observe(End_loan_eventhandler, names = 'value')


#display(New_owner)
#display(squads)
#display(cost_num)

#display(End_season)
#display(fee_num)
#display(Redeemed)

btn_head_par = widgets.Button(description='Generic Parameters',layout=widgets.Layout(width='300px', height='40px'))
btn_head_par_loan = widgets.Button(description='Loan Parameters',layout = btn_head_par.layout)
btn_head_par.style.button_color = 'skyblue'
btn_head_par_loan.style.button_color = 'skyblue'



left_box = widgets.VBox([btn_head_par,New_owner, squads, cost_num])
right_box = widgets.VBox([btn_head_par_loan,End_season, fee_num, Redeemed])
input_par = widgets.HBox([left_box, right_box])
display(input_par)


## Click button to show exchange options

In [7]:
btn_exch_par = widgets.Button(description='Exchange Parameters',layout=widgets.Layout(width='350px', height='40px'))
btn_exch_par.style.button_color = 'orange'

def exch_par_eventhandler(obj):

    list_ow = owner_players(new_owner)
    list_ow.insert(0,'Choose one')
    Ex_players = widgets.Dropdown(options = list_ow, description = 'Exch. Player')

    def Ex_pl_eventhandler(change):

        global exch_pl
        exch_pl = change.new
        print('If exchange is chosen, '+pl.name+' will be exchanged with ' + exch_pl+'              ', end="\r", flush=True)

    Ex_players.observe(Ex_pl_eventhandler, names = 'value')

    #display(Ex_players)

    Ex_squads = widgets.Dropdown(options = ['Choose one','main', 'primavera'], description = 'Exch. Squad')

    def squad_eventhandler(change):
        global squad_ex_pl
        squad_ex_pl = change.new
        print('After operation '+exch_pl+' will be in ' + G_owner+'\'s '+squad_ex_pl+' squad                ', end="\r", flush=True)

    Ex_squads.observe(squad_eventhandler, names = 'value')

    #display(Ex_players)
    #display(Ex_squads)

    btn_head_exch = widgets.Button(description='Select Exchange Player',layout=widgets.Layout(width='350px', height='40px'))

    btn_head_exch.style.button_color = 'skyblue'
    
    btn_conf_ex_par = widgets.Button(description='Confirm Parameters')
    
    def conf_par_eventhandler(obj):
        clear_output()
        print(pl.name+' will be exchanged with ' + exch_pl+'. He will play in '+ G_owner+'\'s '+squad_ex_pl+' squad                ', end="\r", flush=True)
        display(btn_exch_par)
    
    
    btn_conf_ex_par.on_click(conf_par_eventhandler)

    input_ex = widgets.VBox([btn_head_exch,Ex_players,Ex_squads, btn_conf_ex_par])
    display(input_ex)
display(btn_exch_par)
btn_exch_par.on_click(exch_par_eventhandler)

Button(description='Exchange Parameters', layout=Layout(height='40px', width='350px'), style=ButtonStyle(butto…

## Remember: Confirm parameters before performing operation

In [8]:
btn_confirm_parameters = widgets.Button(description='Confirm Parameters',layout=widgets.Layout(width='603px', height='40px'))

def confirm_parameters_event(obj):
    
    btn_draft = widgets.Button(description='Draft Player')
    
    def draft_event(obj): 
        
        check = pl.draft_in(new_owner, squad = squad, cost = cost)
        clear_output()
        if check:
            print(pl.name + ' was drafted by '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.               ', end="\r", flush=True)
        else:
            print('Unsufficient Funds!! Operation was canceled')
        display(btn_confirm_parameters)
    btn_draft.on_click(draft_event)

    btn_auction = widgets.Button(description='Auction Player')
    
    def auction_event(obj):
        
        check = pl.auction_in(new_owner, squad = squad, cost = cost)
        clear_output()
        if check:
            print(pl.name + ' was adjudicated by '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.                ', end="\r", flush=True)
        else:
            print('Unsufficient Funds!! Operation was canceled')
        display(btn_confirm_parameters)
    btn_auction.on_click(auction_event)

    btn_cash = widgets.Button(description='Buy Player')
    
    def cash_event(obj):
        
        check = pl.cash_in(new_owner, squad = squad, cost = cost)
        clear_output()
        if check:
            print(pl.name + ' was purchased by '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.                    ', end="\r", flush=True)
        else:
            print('Unsufficient Funds!! Operation was canceled')
        display(btn_confirm_parameters)
        
        
    btn_cash.on_click(cash_event)

    btn_exch = widgets.Button(description='Exchange Players')
    
    def exch_event(obj):
        if cost > 0:
            payer = new_owner
        else:
            payer = G_owner
        
        check = pl.exchange_with(exch_pl, cost, squad, squad_ex_pl)
        clear_output()
        
        if check:
            print(pl.name + ' was exchanged with '+ exch_pl+'. '+payer+' paid '+str(abs(cost))+' FM.                         ', end="\r", flush=True)
        else:
            print('Unsufficient Funds!! Operation was canceled')
        display(btn_confirm_parameters)
    btn_exch.on_click(exch_event)

    btn_loan = widgets.Button(description='Start Loan')
    
    def loan_event(obj):
        
        check = pl.loan_to(new_owner, cost, squad, end_season, final_fee)
        clear_output()
        if check:
            print(pl.name + ' was sent on loan to '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.                        ', end="\r", flush=True)
        else:
            print('Unsufficient Funds!! Operation was canceled')
        display(btn_confirm_parameters)
    btn_loan.on_click(loan_event)


    btn_loan_end = widgets.Button(description='End Loan')
    
    def end_loan_event(obj):
        
        pl.end_loan(squad, redeemed)
        clear_output()
        if redeemed is True:
            print('Loan of '+pl.name + ' was terminated. Will be redeemed: Register separate transaction.                    ', end="\r", flush=True)
        else:
            print('Loan of '+pl.name + ' was terminated. Will NOT be redeemed.                         ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_loan_end.on_click(end_loan_event)

    btn_move_up = widgets.Button(description='Move Up')
    
    def move_up_event(obj):
        
        pl.move_up()
        clear_output()
        print(pl.name + ' was moved to main squad.                                     ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_move_up.on_click(move_up_event)
    
    btn_rescind = widgets.Button(description='Rescind')
    
    def rescind_event(obj):
        
        pl.rescind(cost)
        clear_output()
        print(pl.name + '\'s contract was rescinded.                                     ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_rescind.on_click(rescind_event)
    
    
    #display(btn_draft)
    #display(btn_auction)
    #display(btn_cash)
    #display(btn_exch)
    #display(btn_loan)
    #display(btn_loan_end)
    #display(btn_move_up)
    
    btn_head_1pl = widgets.Button(description='1 pl Ops.')
    btn_head_2pl = widgets.Button(description='Exchanges')
    btn_head_loans = widgets.Button(description='Loans')
    btn_head_prom = widgets.Button(description='Promotions')
    
    btn_head_1pl.style.button_color = 'skyblue'
    btn_head_2pl.style.button_color = 'skyblue'
    btn_head_loans.style.button_color = 'skyblue'
    btn_head_prom.style.button_color = 'skyblue'  
    
    box1 = widgets.VBox([btn_head_1pl,btn_draft, btn_auction, btn_cash,btn_rescind])
    box2 = widgets.VBox([btn_head_2pl,btn_exch])
    box3 = widgets.VBox([btn_head_loans,btn_loan, btn_loan_end])
    box4 = widgets.VBox([btn_head_prom,btn_move_up])
    input_final = widgets.HBox([box1, box2, box3, box4])
    display(input_final)
    
    
btn_confirm_parameters.style.button_color = 'lightgreen'   
btn_confirm_parameters.on_click(confirm_parameters_event) 

display(btn_confirm_parameters)

Button(description='Confirm Parameters', layout=Layout(height='40px', width='603px'), style=ButtonStyle(button…

# Left to do:

- Save historic stats, but if we save the entire collection this becomes not necessary, as we can extract historic stats from there

